# Binary Painting Classification #

In this notebook we implement a convolutional autoencoder for binary classification of paintings by painter. This classifier serves as a proof of concept for our convolutional autoencoder classifier.

In [2]:
# import modules
from utils.image_scrape import *
from utils.image_formatting import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from PIL import Image

%load_ext autoreload
%autoreload 2

Download 1000 picasso and 1000 van gogh images.

In [5]:
# downloading data for only two artists. Trying a binary classification first
im_per_artist = 1000
artists = {}
artists.update({ 'pablo-picasso' : PAINTER_DICT['pablo-picasso'] })
artists.update({ 'vincent-van-gogh' : PAINTER_DICT['vincent-van-gogh'] })

resolution = (750, 750)
download_data(artists=artists, images_per_artist=im_per_artist)
standardize_images(resolution, artists=artists)

Load images into memory as numpy arrays.

In [45]:
gogh_images = get_images('vincent-van-gogh', 500)
picasso_images = get_images('pablo-picasso', 500)

print(np.shape(picasso_images))
print(np.shape(gogh_images))

(500, 750, 750, 3)
(500, 750, 750, 3)


Make labels. Picasso = 0, Gogh = 1

In [46]:
y = []
X = []
for picasso, gogh in zip(picasso_images, gogh_images):
    y.append(0)
    y.append(1)
    X.append(picasso)
    X.append(gogh)

print(len(X), len(y))

1000 1000


Shuffle and split into train and validation sets

In [47]:
zipped = list(zip(X, y))            # zip to shuffle and retain correct labels
np.random.shuffle(zipped)           # shuffle zipped batch
X, y = zip(*zipped)
split_index = int(0.8 * len(X))
X_train = np.array(X[:split_index], dtype=np.float32)
y_train = np.array(y[:split_index], dtype=np.float32)
X_val = np.array(X[split_index:], dtype=np.float32)
y_val = np.array(y[split_index:], dtype=np.float32)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(800, 750, 750, 3) (800,)
(200, 750, 750, 3) (200,)


In [48]:
output_size = max(y_train) + 1  # define output size of network

input_shape = (X_train[0].shape[0], X_train[0].shape[1], X_train[0].shape[2]) # this should NOT take into account batches. Keras adds batch dimensions

# define function to generate model
def create_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(filters=6, kernel_size=(5, 5), strides=(1,1), activation='tanh', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(filters=6, kernel_size=(5, 5), strides=(1,1), activation='tanh', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(Dense(400, activation='relu'))
    model.add(Dense(output_size, activation='softmax'))
    
    return model

In [49]:
model_test = create_model()  # create a model

model_test.compile(optimizer='adam',  # compile model
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_test.fit(x=X_train, # fit model
          y=y_train,
          batch_size=64,
          epochs=10, 
          validation_data=(X_val, y_val), 
          callbacks=None,
          verbose=1)

Epoch 1/10
13/13 [==============================] - 181s 14s/step - loss: 67.6477 - accuracy: 0.5487 - val_loss: 17.4725 - val_accuracy: 0.5600
Epoch 2/10
13/13 [==============================] - 170s 13s/step - loss: 10.7067 - accuracy: 0.7100 - val_loss: 8.2917 - val_accuracy: 0.6400
Epoch 3/10
13/13 [==============================] - 183s 14s/step - loss: 4.7367 - accuracy: 0.7287 - val_loss: 2.1466 - val_accuracy: 0.6950
Epoch 4/10
13/13 [==============================] - 165s 13s/step - loss: 1.1759 - accuracy: 0.8000 - val_loss: 3.2649 - val_accuracy: 0.6200
Epoch 5/10
13/13 [==============================] - 167s 13s/step - loss: 0.5740 - accuracy: 0.8875 - val_loss: 2.3795 - val_accuracy: 0.6700
Epoch 6/10
13/13 [==============================] - 167s 13s/step - loss: 0.3519 - accuracy: 0.9150 - val_loss: 1.7484 - val_accuracy: 0.7400
Epoch 7/10
13/13 [==============================] - 165s 13s/step - loss: 0.1429 - accuracy: 0.9600 - val_loss: 1.4599 - val_accuracy: 0.7400
Epo

In [7]:
model_path = "models/binary_model_{}_{}".format(list(artists.keys())[0], list(artists.keys())[1]) # save model
model_test.save(model_path)

Predict on single images to get a feel for model quality. Picasso : 0, Van Gogh : 1

In [15]:
image_to_predict = get_images('vincent-van-gogh', 10)  # get 10 random van gogh images
# Image.fromarray(image_to_predict[0]).show()
loaded_model = tf.keras.models.load_model(model_path)  # load model
print(np.argmax(loaded_model.predict(image_to_predict), axis=1)) # predict

[1 0 1 0 1 1 1 1 1 1]


The naive binary classification model performs well, although the model appears to be overfitting. More sophisticated models, with regularization, will be tested in the future. 